In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.measure import block_reduce
import pickle
import torch

from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression
from scipy.ndimage import gaussian_filter as gf
from torch.nn.functional import pad, one_hot



In [2]:
input_data = ["CMIP_ews_CMCC-CESM_Processed.pkl","CMIP_ews_CSIRO-Mk3-6-0_Processed.pkl","CMIP_ews_FGOALS-g2_Processed.pkl","CMIP_ews_GFDL-ESM2M_Processed.pkl"]
data_list = []
labels_list = []


for name in input_data:
    with open(name,"rb") as file:
        raw_data = pickle.load(file)
        seq = [torch.from_numpy(run).float() for run in raw_data["x"]]
        seq_padded = []
        max_length = 600
        for run in seq:
            pad_amount = max_length - run.shape[0]
            run_padded = pad(run, (0, 0, pad_amount, 0))
            seq_padded.append(run_padded)

        input_data = torch.stack(seq_padded)
        data_list.append(input_data)

        labls = [torch.tensor(label) for label in raw_data["null"]]
        labels_t = torch.stack(labls)
        labels_oh = one_hot(labels_t).float()
        labels_list.append(labels_oh)

data = torch.concat(data_list)
labels = torch.concat(labels_list)

In [3]:
data.shape, labels.shape

(torch.Size([62511, 600, 12]), torch.Size([62511, 2]))

In [4]:
labels[100]

tensor([0., 1.])

In [ ]:
# seq_trunc = []
# for run in seq:
#     t = 0
#     while torch.equal(run[t,:],torch.zeros(12)):
#         t += 1
#         seq_trunc.append(run[t:,:])

In [5]:
torch.save(data,"Test_Data")
torch.save(labels,"Test_Labels")